### (1) Imports, paths, and config

In [ ]:
import sys
import os
from os import environ
from pathlib import Path

import torch
import numpy as np
import matplotlib.pyplot as plt

from hydra import initialize, compose
from hydra.utils import instantiate
from omegaconf import DictConfig, OmegaConf

from lensing_catalog import LensingTileCatalog
from lensing_variational_dist import LensingVariationalDistSpec, LensingVariationalDist

In [ ]:
environ["BLISS_HOME"] = "/home/twhit/bliss"
with initialize(config_path=".", version_base=None):
    cfg = compose("lensing_config", overrides={"surveys.sdss.load_image_data=true"})

### (2) Generate synthetic images

In [ ]:
simulator = instantiate(cfg.simulator)
batch = simulator.get_batch()

In [ ]:
_ = plt.imshow(batch['images'][0][2])

### (3) Instantiate encoder

In [ ]:
encoder = instantiate(cfg.train.encoder)
target_cat = LensingTileCatalog(encoder.tile_slen, batch["tile_catalog"])
truth_callback = lambda _: target_cat

### (4) Train encoder on a single batch of synthetic images

Here we'll just try to learn the shear and convergence for the single batch of synthetic images we generated above.

Later, we'll generate a new batch inside the training loop in each iteration.

In [ ]:
num_iters = 1000
optimizer = torch.optim.Adam(encoder.parameters(), lr=1e-3)

for i in range(num_iters):
    # Forward pass of encoder
    pred = encoder.infer(batch, truth_callback)
    lvd = pred['marginal']
    
    # Compute loss and take optimizer step
    optimizer.zero_grad()
    loss = lvd.compute_nll(target_cat).mean()
    loss.backward()
    optimizer.step()
    
    if i % 10 == 0:
        print('Iteration {}: Loss {}'.format(i, loss.item()))

### (5) Summarize results

In [ ]:
batch["tile_catalog"]["shear"].squeeze()[0][:,:,1]

In [ ]:
q = lvd.factors
q["shear"].mean[0][:,:,1]

In [ ]:
cfg.prior.convergence_a/(cfg.prior.convergence_a + cfg.prior.convergence_b)

In [ ]:
batch["tile_catalog"]["convergence"].squeeze()[0]

In [ ]:
q["convergence"].base_dist.mean.sigmoid()[0]